In [1]:
import os, sys
import csv
from collections import defaultdict
from datetime import datetime
import json

sys.path.insert(0, "/userdata/sjain/ECoG_data_collection/")
from ecog_preprocessing.transformData import transform as transformData
sys.path.insert(0, "/userdata/sjain/ECoG_data_collection/stimulus_align/src")
import preanalysis

In [3]:
datapath = "/data_store1/human/prcsd_data/"
savepath = "data/preprocessed"

subj_blocks = defaultdict(list)
bad_combos = []
good_rows = []

with open("task_database_logging/task_database_Info_20240313.csv", newline="") as csvfile:
    reader = csv.DictReader(csvfile)
    rows = [row for row in reader]
# if os.path.exists("task_database_logging/task_database_Log.json"):
#     with open("task_database_logging/task_database_Log.json", "r") as f:
#         LOAD = json.load(f)
#     done_blocks = [f"{db['SubjectID']}_B{db['Block']}" for db in LOAD]

In [17]:
# Read each entry from our task database.
for row in rows:
    subj, block, task, valid = row["SubjectID"], row["Block"], row["Task"], row["Valid"]
    subj_id = int(subj[2:])
    if valid.lower()=="bad" or task not in ["TIMIT", "MOCHA"] or subj_id<id_start or subj_id>=id_end: continue
    subj_blocks[subj].append(block)

    blockname = f"{subj}_B{block}"
    blockpath = os.path.join(datapath, subj, blockname)
    storepath = os.path.join(savepath, subj, blockname)
    
    d1_path = os.path.join(storepath, f"{blockname}_nocar_Hilb.h5")
    d2_path = os.path.join(storepath, "ecog400/ecog.mat")
    if os.path.exists(d1_path) and os.path.exists(d2_path):
        print(f"{blockname} was processed already!")
        if not os.path.exists(d1_path.replace("_nocar", "_100Hz_nocar")):
            allband = preanalysis.preanalyze_data_h5(subj, block, 100, return_ds=True)
        continue
    try:
        print(f"Processing blockname {blockname}.\n")
        transformData(blockpath, rate=400, car=False, line_noise_notch=True, force=True, storepath=storepath)
        preanalysis.preanalyze_data_h5(subj, block, 100, storepath)
        # If able to preprocess, record the entire `row` as good.
        row["prcsd_date"] = datetime.today().strftime('%Y-%m-%d %H:%M:%S')
        row["prcsd_loc"] = storepath
        good_rows.append(row)
        with open(f"{storepath}/preproc_log.json", "w") as f:
            json.dump(good_rows, f)
    except FileNotFoundError:
        print(f"Didn't find data. Skipping blockname {blockname}.\n")
        # If not able to preprocess, record the `subject-block` as bad.
        bad_combos.append(blockname)

Processing blockname EC6_B36.

do downsampling
Didn't find data. Skipping blockname EC6_B36.

Processing blockname EC6_B38.

do downsampling
Didn't find data. Skipping blockname EC6_B38.

Processing blockname EC6_B42.

do downsampling
Didn't find data. Skipping blockname EC6_B42.

EC28_B3 was processed already!
Processing blockname EC28_B4.

do downsampling
Didn't find data. Skipping blockname EC28_B4.

Processing blockname EC28_B6.

do downsampling


applying Hilbert transform: 100%|██████████| 40/40 [18:19<00:00, 27.49s/it]
/userdata/sjain/ECoG_data_collection/ecog_preprocessing/transformData.py:280: H5pyDeprecationWarning: other_ds.dims.create_scale(ds, name) is deprecated. Use ds.make_scale(name) instead.
  dset.dims.create_scale(f[name], name)


Reading h5 files
Resampling


/userdata/sjain/myenv/lib/python3.6/site-packages/scipy/stats/stats.py:2497: RuntimeWarning: Mean of empty slice.
  mns = a.mean(axis=axis, keepdims=True)
/userdata/sjain/myenv/lib/python3.6/site-packages/numpy/core/_methods.py:163: RuntimeWarning: invalid value encountered in true_divide
  ret, rcount, out=ret, casting='unsafe', subok=False)
/userdata/sjain/myenv/lib/python3.6/site-packages/numpy/core/_methods.py:234: RuntimeWarning: Degrees of freedom <= 0 for slice
  keepdims=keepdims)
/userdata/sjain/myenv/lib/python3.6/site-packages/numpy/core/_methods.py:195: RuntimeWarning: invalid value encountered in true_divide
  arrmean, rcount, out=arrmean, casting='unsafe', subok=False)
/userdata/sjain/myenv/lib/python3.6/site-packages/numpy/core/_methods.py:224: RuntimeWarning: invalid value encountered in true_divide
  ret, rcount, out=ret, casting='unsafe', subok=False)


Saving downsampled data
Processing blockname EC28_B7.

do downsampling


applying notch filters:   0%|          | 0/3 [00:06<?, ?it/s]


KeyboardInterrupt: 